# Preparación para el TF MDS

## Preparación de los datos

Repo para trabajo de preparación

Dataset: https://www.yelp.com/dataset/documentation/main

Crea un repositório:
- con una única carpeta
- 100 fotos de cada categoría (label)
- fotos escaladas y recortadas a 100 x 100

**Requiere ejecución del notebook '00_EDA.ipynb'** para generar el archivo checkpoints/df5.pkl

In [1]:
import os
import pickle
import pandas as pd
from PIL import Image
from tqdm.notebook import trange, tqdm
import torchvision.transforms as T

In [4]:
dataset_photos_path = 'dataset/photos'
dataset_output_path = 'dataset/toy_flat'
photo_size = 100 # 100 x 100 pixeles
df_pickle_file='checkpoints/df5.pkl'
max_photos=100

In [5]:
# crear carpeta para el repositorio o vaciarla si ya existe
if os.path.isdir(dataset_output_path): # carpeta de salida ya existe. Se elimina (vacia)
    shutil.rmtree(dataset_output_path, ignore_errors=False,
                  onerror=None)  # remove tree
os.mkdir(dataset_output_path)

In [ ]:
# Cargar fichero
df = pickle.load(open(df_pickle_file,'rb'))

In [ ]:
# Seleccionar fotos a usar
if max_photos > 0: # max_photos == 0 significa usar todas las fotos
    df_subset = pd.DataFrame([], columns=['photo_id',
                                    'label',
                                    'x_dim', 
                                    'y_dim', 
                                    'z_channels', 
                                    'pixels', 
                                    'drink',
                                    'food',
                                    'inside',
                                    'menu',
                                    'outside'])
    total_photos = df.label.value_counts()
    for label in total_photos.index:
        if total_photos[label] > max_photos: # Hay muchas fotos. Preservar un subconjunto.
            df_subset = pd.concat([df_subset,
                                df.loc[df.label == label].sample(n=max_photos)])
        else: # Hay pocas fotos. Preservar todas.
            df_subset = pd.concat([df_subset,
                                df.loc[df.label == label]])
    df = df_subset
    del(df_subset)

In [ ]:
# redimensionar, recortar y guardar fotos

for img in tqdm(range(len(df)), desc='Photos cropped', miniters=len(df)//100):
    im = Image.open('{}/{}.jpg'.format(dataset_photos_path, df.iloc[img].photo_id))

    # redimensionar
    if (df.iloc[img].y_dim < df.iloc[img].x_dim): # imagen estrecha
        width = int(photo_size)
        height = math.floor(photo_size * df.iloc[img].x_dim/df.iloc[img].y_dim)
    else: # imagen ancha
        width = math.floor(photo_size * df.iloc[img].y_dim/df.iloc[img].x_dim)
        height = int(photo_size)

    resized = T.Resize((height, width))(im)
    cropped = T.CenterCrop(size=int(photo_size))(resized)

    cropped.save('{}/{}/{}.png'.format(
        dataset_output_path, 
        df.iloc[img].label, 
        df.iloc[img].photo_id))

In [ ]:
pickle.dump(df,open('{}/df.pkl'.format(dataset_output_path,'wb'))

In [ ]:
from email.mime import image
import math
